# Emulate dlib network with a pretrained smaller Resnet

### Pytorch dep loading

In [3]:
from __future__ import print_function,division
import torch
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo

import matplotlib as plt
from graphviz import Digraph
import re
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
import numpy as np
print('dep loaded')

dep loaded


### Prepare dataset

In [12]:
import skimage
from skimage import io,data
import glob
import cv2
import dlib
import csv
import scipy.io
import os


def getImageList(completePath):
    image_list = []
    print('Loading imgs from:' + completePath)
    for root,dirs,files in os.walk(completePath):
        #for filename in glob.glob(completePath + '/*.jpg'): #assuming gif
        for filename in files:
            if filename.endswith((".jpg",".jpeg",".png",".JPG")):
                fullName = str(root) + '/' + str(filename)
                #print(fullName)
                image_list.append(fullName)
    print('done loading ' + str(len(image_list)) + " imgs")
    return image_list

def genOneList(dlibHogdetector,facerec,predictor,imagefullpath,savepath,NDIM,name):

    image_list = getImageList(imagefullpath)
    #print(image_list)
    #descrp_Vector = np.zeros((len(image_list),NDIM))
    descrp_Vector_dlib = []

    ok_image_lst = []
    ii = 0
    for fn in image_list:
        im = cv2.imread(fn)
        #print(fn,im.shape)
        if len(im.shape) == 3 and im.shape[2] > 1:
            if im.shape[2] > 3:
                im = im[:,:,0:3]
                print("reshaped",im.shape)
            imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        else:
            imgray = im
            #create colour image
            im = np.zeros((im.shape[0],im.shape[1],3))
            im[:,:,0] = imgray
            im[:,:,1] = imgray
            im[:,:,2] = imgray
        
        # usa face detector
        faces, FaceScores, Faceidx = dlibHogdetector.run(imgray,1,0.0) #frame, piramidal level, face threshold

        faceArea = []
        xar = []
        yar = []
        war = []
        har = []
        facesobj = dlib.full_object_detections() # array of full_object_detection objects
        for k, d in enumerate(faces):
            x = d.left()
            y = d.top()
            w = d.right() - x 
            h = d.bottom() - y
            faceArea.append(w*h)
            xar.append(x)
            yar.append(y)
            war.append(w)
            har.append(h)

        if len(faceArea) > 0: #dlib has detected at least a face
            #faceArea = np.asarray(faceArea)
            bestFaceIdx = np.argmax(faceArea)
            dlibFaceRect = dlib.rectangle(int(xar[bestFaceIdx]),
                                                int(yar[bestFaceIdx]),
                                                int(xar[bestFaceIdx] + war[bestFaceIdx]),
                                                int(yar[bestFaceIdx] + har[bestFaceIdx]))
            shape = predictor(imgray, dlibFaceRect) #landmark detector on dlibFaceRect
            facesobj.append(shape) #landmark detector
            images = dlib.get_face_chips(im, facesobj, size=80, padding=0.0)          
            cv2.imwrite(str(savepath) + 'imgs/img_' + name + '_'+ str(ii) + "_" + '.png',images[0])
            descrp_Vector_dlib.append(facerec.compute_face_descriptor(im, shape, 1))
            ii += 1
            ok_image_lst.append(fn)


    #assert(len(ok_image_lst) == len(descrp_Vector_openface) == len(descrp_Vector_dlib))
    print("Face descriptors for:" + str(len(ok_image_lst)) + ' images')
    ok_image_lst = np.asarray(ok_image_lst)
    descrp_Vector_dlib = np.asarray(descrp_Vector_dlib)
    assert(descrp_Vector_dlib.shape[0] == len(ok_image_lst))
    assert(descrp_Vector_dlib.shape[1] == NDIM)

    print('Writing files')
    #text_file = open(str(savepath) + name + "_fnames.txt", "w")
    np.save(str(savepath)+name+"_feat.npy", descrp_Vector_dlib)
    np.save(str(savepath)+name+"_fname.npy", ok_image_lst)
    #for line in ok_image_lst:
    #    text_file.write(line + "\n")
    #text_file.close()

#### START ###
NDIM = 128
OUTIMGSIZE = 80

completePath = "/mnt/mydata/dataset_faces/lfwa/"
savepath = "/mnt/mydata/dataset_faces/precomputed_feats/"
dlibshapePred_path = '../../SmartVend/FaceDet/objDetectors/landmarkDet/shape_predictor_68_face_landmarks.dat'
dlibFaceRecpath = '../../SmartVend/FaceDet/faceRecmodels/dlib_face_recognition_resnet_model_v1.dat'

dlibHogdetector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(dlibshapePred_path)
facerec = dlib.face_recognition_model_v1(dlibFaceRecpath)


dirlist = []
for a,b,c in os.walk(completePath):
    dirlist = b
    break
print(dirlist)
for name in dirlist:
    genOneList(dlibHogdetector,facerec,predictor,completePath+name+'/',savepath,NDIM,name)

print('saved all')



['lfw2']
Loading imgs from:/mnt/mydata/dataset_faces/lfwa/lfw2/
done loading 13233 imgs
Face descriptors for:13145 images
Writing files


### Pretrained resnet class

In [3]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model


### Load the pretrained resnet, modify it for any input size and 128 dim output vector

In [20]:
# laod an imagenet pretrained network
net = resnet18(pretrained=True,num_classes=1000)
#print(net)

#modify the net
net.avgpool = nn.AdaptiveAvgPool2d(1)
net.fc = nn.Linear(512,128)

x = torch.rand(1,3,80,80)
y = net(x)
print(y.shape)

torch.Size([1, 128])


#### old  testcode

In [ ]:
%matplotlib inline
import matplotlib as plt
from graphviz import Digraph
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models

from graphviz import Digraph
# make_dot was moved to https://github.com/szagoruyko/pytorchviz
from torchviz import make_dot
from torchviz import make_dot, make_dot_from_trace



x = torch.randn(1,3,224,224)
resnet18 = models.resnet18()
print(resnet18.type)
modules = list(resnet18.children())[:-1]
resnet18 = nn.Sequential(*modules)
print(resnet18.type)
resnet18.fc = nn.Linear(512 * block.expansion, 128)
print(resnet18.type)
#print(resnet18)
y = resnet18(Variable(x))
print(y.shape)

g = make_dot(y, params=dict(list(resnet18.named_parameters()) + [('x', x)]))
print('end')